In [ ]:
# Importing necessary modules
import seaborn as sns
import pyspark.sql.functions as F
import pandas as pd
import matplotlib.pyplot as plt
from operator import add
from functools import reduce
import numpy as np
import re
import os
from pyspark.sql.types import StructField, StructType, StringType, LongType
from pyspark.sql.functions import *
import random

# Setting up visualization
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

23/03/17 18:57:02 WARN org.apache.spark.scheduler.cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/03/17 18:57:17 WARN org.apache.spark.scheduler.cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [ ]:
# These are the states that do not have strict voter ID laws:
#  'VM2Uniform--CA--2021-05-02',	VM2Uniform--CA--2021-05-02	CA	x	California
#  'VM2Uniform--IL--2021-03-05',	VM2Uniform--IL--2021-03-05	IL	x	Illinois
#  'VM2Uniform--MA--2021-01-19',	VM2Uniform--MA--2021-01-19	MA	x	Massachusetts
#  'VM2Uniform--MD--2021-02-15',	VM2Uniform--MD--2021-02-15	MD	x	Maryland
#  'VM2Uniform--ME--2021-05-28',	VM2Uniform--ME--2021-05-28	ME	x	Maine
#  'VM2Uniform--MN--2021-02-14',	VM2Uniform--MN--2021-02-14	MN	x	Minnesota
#  'VM2Uniform--NC--2021-05-18',	VM2Uniform--NC--2021-05-18	NC	x	North Carolina
#  'VM2Uniform--NE--2021-01-20',	VM2Uniform--NE--2021-01-20	NE	x	Nebraska
#  'VM2Uniform--NJ--2021-03-11',	VM2Uniform--NJ--2021-03-11	NJ	x	New Jersey
#  'VM2Uniform--NM--2021-02-25',	VM2Uniform--NM--2021-02-25	NM	x	New Mexico
#  'VM2Uniform--NV--2021-06-13',	VM2Uniform--NV--2021-06-13	NV	x	Nevada
#  'VM2Uniform--NY--2021-03-15',	VM2Uniform--NY--2021-03-15	NY	x	New York
#  'VM2Uniform--OR--2021-02-05',	VM2Uniform--OR--2021-02-05	OR	x	Oregon
#  'VM2Uniform--PA--2021-05-20',	VM2Uniform--PA--2021-05-20	PA	x	Pennsylvania
#  'VM2Uniform--VT--2021-05-28',	VM2Uniform--VT--2021-05-28	VT	x	Vermont

# For each of these states, I want to pull enough samples to get a total sample of 1/2 M; can increase later

# grab files
states =  [
# For now, just exclude New York and Califonria, because the parquet files take too long to read
 'VM2Uniform--VT--2021-05-28'
    
    
    
#  'VM2Uniform--IL--2021-03-05',
#  'VM2Uniform--MA--2021-01-19',
#  'VM2Uniform--MD--2021-02-15',
#  'VM2Uniform--ME--2021-05-28',
#  'VM2Uniform--MN--2021-02-14',
#  'VM2Uniform--NC--2021-05-18',
#  'VM2Uniform--NE--2021-01-20',
#  'VM2Uniform--NJ--2021-03-11',
#  'VM2Uniform--NM--2021-02-25',
#  'VM2Uniform--NV--2021-06-13',
#  'VM2Uniform--OR--2021-02-05',
#  'VM2Uniform--PA--2021-05-20',
 
    
     #'VM2Uniform--CA--2021-05-02',

     #'VM2Uniform--NY--2021-03-15',

]

gcs_path = 'gs://pstat135-voter-file/VM2Uniform'
numrows = dict()

# first DF
df_ref = spark.read.parquet('gs://pstat135-voter-file/VM2Uniform/')
df_ref = df_ref.sample(True, percentage_sample, seed = 19480384)

23/03/17 19:38:17 WARN org.apache.spark.scheduler.cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/03/17 19:38:32 WARN org.apache.spark.scheduler.cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/03/17 19:38:47 WARN org.apache.spark.scheduler.cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/03/17 19:39:02 WARN org.apache.spark.scheduler.cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/03/17 19:39:17 WARN org.apache.spark.scheduler.cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are regist

In [ ]:
df_ref.show(10)

23/03/17 18:56:17 WARN org.apache.spark.scheduler.cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [ ]:
percentage_sample = 0.0098788 # 500000 / 50613433

In [ ]:
df_ref.show(10)

23/03/17 18:39:32 WARN org.apache.spark.scheduler.cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/03/17 18:39:47 WARN org.apache.spark.scheduler.cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/03/17 18:40:02 WARN org.apache.spark.scheduler.cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/03/17 18:40:17 WARN org.apache.spark.scheduler.cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/03/17 18:40:32 WARN org.apache.spark.scheduler.cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are regist

In [ ]:
# others
for one_state in states[1:]:
    print("%s: " % (one_state), end="")
    
    # read dataframe for one_state
    df = spark.read.parquet("/".join([gcs_path, one_state]))
    
    # sample small proportion
    sample_df = df.sample(True, percentage_sample, seed = 19480384)
    
    # Then, rbind / concatenate all the files together
    df_ref = df_ref.union(sample_df)

# Save as parquet into google cloud

df_ref.write.format("parquet").save("total_reference_sample")
df_ref = spark.read.parquet("total_reference_sample")

# Then, run the Indiana file's code on them